In [1]:
# Sampling
# 1. Sample using HGT Sampler as outlined in the paper, using pyg implementations
# 2. The sampling is adapted to link prediction, by first sampling random supervision edges of which the nodes create the supervision nodes
# a. Dataset is divided across multiple dimensions:
#   a.1. Split into Train, Val, Test split (96, 2, 2)
#   a.2. Training only: Edges are split into those which are used solely for message passing and those solely used for supervision (80, 20). 
#        Because an expressive model (HGT) is used, this prevents the model from memorizing supervision edges by their appearance as message passing edges
#   a.3. This means Training consists of 96%*80% Message passing Edges, 96%*20% supervision edges, Val contains 2% Supervision Edges, Test contains 2% supervison Edges
#   a.4. Validation and Test edges use the Training Message passing Edges as well.
# b. For mini-batch sampling in the training phase, first x random edges are sampled as supervision edges. 
#    For the nodes of these supervision edges, we apply batch-wise HGT Sampling. Due to implementation limitations, for each supervision entity type, the hgt sampling is separate. 
#    This limitation does not apply for sampled neighbor entity types
# during sampling, also the reverse edge of the supervision edge is removed to avoid data leakage


# HGT Sampler (See Paper for further reference)
# The probablity of a neighbor node s to be sampled depends on the normalized degree of all its edge types connecting it to all source nodes
# If neighbor node s is connected to a and b by edge type r, and a has 2 neighbors through edge type r and b has 1 neighbor (node s) through edge type r, 
# then the sampling probablity of s is (1/2+1)**2 / 2**2, if it were connected through other edge types to the nodes as well, those degrees would be added to the numerator and denominator.
# Nodes are sampled without replacement.
# This sampling strategy creates more dense mini-batches, because neighbor nodes which are connected to multiple source nodes and by multiple relationship types are sampled more frequently.
# Therefore, training is sped up since less node representations have to be computed. Furthermore, as stated in the paper, the sampling method allows to sample a 
# similar number of neighbors for each edge type, because high-count edge types and low-count edge types are weighted equally. For each neighbor node type T, a fixed number n of nodes is sampled.




In [2]:
#%load_ext autoreload
#%autoreload 2

In [3]:
# sampler for Heterogeneous Graph Transformer

In [4]:
# for each node type, add a new edge type only consisting of self loops
# this is done to allow HGT to attend to the previous node representations
# for node_type in data.node_types:
#     data[node_type, 'self_loop', node_type] = torch.cat((torch.arange(data[node_type].num_nodes),torch.arange(data[node_type].num_nodes)), dim=0)

In [5]:
# split

In [11]:
import os
import torch
from torch_geometric.data import HeteroData

def get_datasets(get_edge_attr=False, filename=None):
    if filename is None:
        filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
    size = os.path.getsize(filename)
    print('size of dataset on disk: ', size/1e9, 'gb')

    if os.path.exists(filename):
        data = HeteroData.from_dict(torch.load(filename))
        print('loading saved heterodata object')


    from tqdm.auto import tqdm
    def top_k_mask(scores, indices, top_k ):
        # Make sure we are using the GPU
        scores = scores.cuda()
        indices = indices.cuda()
        
        # Create an empty mask with the same shape as scores
        mask = torch.zeros_like(scores, dtype=torch.bool)
        # Get the unique indices and their counts
        unique_indices, counts = torch.unique(indices, return_counts=True)
    
        # Indices where count > top_k
        large_indices = unique_indices[counts > top_k]
    
        # Set mask for indices where count <= top_k
        mask[~torch.isin(indices,large_indices)] = True
        # For indices where count > 50, we only keep top 50 scores
        for idx in tqdm(large_indices):
            idx_mask = (indices == idx)
            values, idxs = scores[idx_mask].topk(top_k)
            a = mask[idx_mask]
            a[idxs] = True
            mask[idx_mask] = a
            
        return mask.cpu()

    
    top_k = 50
    print('for skill job edges keep top k edges per job, k is ',top_k)
    e = ('skills', 'job_skill', 'jobs')
    rev_e = (e[2],'rev_'+e[1],e[0])
    cache_dir = 'cache'
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    
    mask_path = os.path.join(cache_dir, 'mask.pt') 
    
    if os.path.isfile(mask_path):
        mask = torch.load(mask_path)
    else:
        mask = top_k_mask(data[e].edge_attr.squeeze(1), data[e].edge_index[1,:], top_k)
        torch.save(mask, mask_path) 
     
    data[e].edge_attr = data[e].edge_attr[mask]
    data[rev_e].edge_attr = data[rev_e].edge_attr[mask]
    data[e].edge_index = data[e].edge_index[:,mask]
    data[rev_e].edge_index = data[rev_e].edge_index[:,mask]

    
    
    from torch_geometric import seed_everything
    import torch_geometric.transforms as T
    from torch_geometric.utils import sort_edge_index


    
    edge_types = []
    rev_edge_types = []
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            rev_edge_types.append(edge_type)
        else:
            edge_types.append(edge_type)

    transform = T.RandomLinkSplit(
        is_undirected=True,
        edge_types=edge_types,
        rev_edge_types=rev_edge_types,
        num_val=0.02,
        num_test=0.05,
        add_negative_train_samples=False, # only adds neg samples for val and test, neg train are added by LinkNeighborLoader. This means for each train batch, negs. are different, for val and train they stay the same
        neg_sampling_ratio=1.0,
        disjoint_train_ratio=0.3, #  training edges are shared for message passing and supervision
        )

    seed_everything(14)
    # sort by col to speed up sampling later (we can sepcify is_sorted=True in link neighbor loader)
    # we actually dont use the sort, because it seems to mess up things, but have not checked if everything works without sorting, so we leave it here
    def sort_edges(data):
        for edge_type in data.edge_types:
            if 'edge_attr' in data[edge_type].keys():
                data[edge_type].edge_index, data[edge_type].edge_attr = sort_edge_index(data[edge_type].edge_index, data[edge_type].edge_attr, sort_by_row=False) 
            else:
                data[edge_type].edge_index = sort_edge_index(data[edge_type].edge_index, sort_by_row=False) 
        return data
    

    def preprocess(data):
        if not get_edge_attr:
            # delete edge_attr of every edge type
            for edge_type in data.edge_types:
                del data[edge_type].edge_attr 

        # delete all keys for every node type except 'x' (e.g. description and title)
        for node_type in data.node_types:
            keys = list(data[node_type].keys())
            for key in keys:
                if key != 'x':
                    del data[node_type][key]
        return data
    
    
    # change all types to float32 and normalize the triangle columns
    for node_type in data.node_types:
        for i in range(data[node_type].x.shape[1]):
            if data[node_type].x[:,i].max()>5:
                #normalize
                print('normalizing column ', i, ' of node type ', node_type)
                data[node_type].x[:,i] = data[node_type].x[:,i]/data[node_type].x[:,i].max()
        
        data[node_type].x = data[node_type].x.to(torch.float32)
        
    
    
    train_data, val_data, test_data = transform(data)
    #train_data = sort_edges(train_data)
    #val_data = sort_edges(val_data)
    #test_data = sort_edges(test_data)
    train_data = preprocess(train_data)
    val_data = preprocess(val_data)
    test_data = preprocess(test_data)
    
    return train_data, val_data, test_data



In [ ]:


from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.loader import HGTLoader
from torch_geometric.sampler import NegativeSampling

def get_hgt_linkloader(data, target_edge, batch_size, is_training, sampling_mode, neg_ratio, num_neighbors_hgtloader, num_workers, prefetch_factor, pin_memory):
    # first sample some edges in linkNeighborLoader
    # use the nodes of the sampled edges to sample from hgt loader
    
    
    num_neighbors_linkloader = [0]
    #for edge_type in data.edge_types:
    #    num_neighbors_linkloader[edge_type] = [0,0]
    
    negative_sampling = NegativeSampling(
        mode=sampling_mode, # binary or triplet
        amount=neg_ratio  # ratio, like Graphsage # 10
        #weight=  # "Probabilities" of nodes to be sampled: Node degree follows power law distribution
        )
    
    if sampling_mode == 'triplet':
        data[target_edge].edge_label = None
        

    linkNeighborLoader = LinkNeighborLoader(
            data,
            num_neighbors=num_neighbors_linkloader,
            edge_label_index=(target_edge, data[target_edge].edge_label_index), # if (edge, None), None means all edges are considered
        
            neg_sampling=negative_sampling, # adds negative samples
            batch_size=batch_size,
            shuffle=is_training, #is_training
            subgraph_type='directional', # contains only sampled edges
            #drop_last=True,
            num_workers=num_workers,
            #disjoint=True # sampled seed node creates its own, disjoint from the rest, subgraph, will add "batch vector" to loader output
        
            #num_workers=2,
            #prefetch_factor=2
            is_sorted = False,
            pin_memory=pin_memory,
            prefetch_factor=prefetch_factor,
    )
   
   
    def get_hgt(data, input_nodetype, input_mask):
        return next(iter(HGTLoader(
                data,
                # Sample 512 nodes per type and per iteration for 4 iterations
                num_samples=num_neighbors_hgtloader,
                batch_size=input_mask.shape[0],
                input_nodes=(input_nodetype, input_mask),
                num_workers=num_workers,
                pin_memory=pin_memory,
                prefetch_factor=prefetch_factor,
            )))
        
    
    def add_self_loops(data):
        for node_type in data.node_types:
            data[node_type, 'self_loop', node_type].edge_index = torch.arange(data[node_type].num_nodes).repeat(2,1)
        return data 

            
    def get_hgt_with_selfloops(loader):
        
        
        for batch in loader:   
            if sampling_mode=='triplet':      
                # original edge_label_index from the whole data object
                unmapped_batchids = torch.cat((batch[target_edge[0]].src_index,batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index)).unique()
                original_edge_label_nodes = torch.LongTensor(batch[target_edge[0]].n_id[unmapped_batchids])

                #remapping or sorting is not needed, since nodes are sorted, also in the htg batch, the edges will be the same
        
                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src = torch.cat((src,src), dim=1)
                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index),dim=0).unsqueeze(0)
                local_edge_label_index = torch.cat((src, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.shape[0])))
                
            elif sampling_mode=='binary':
                unmapped_batchids = batch[target_edge].edge_label_index.flatten().unique()
                original_edge_label_nodes = torch.LongTensor(batch[target_edge[0]].n_id[unmapped_batchids])

            else:
                raise Exception('binary or triplet sampling mode')
                
                
            hgt_batch = get_hgt(data, target_edge[0], original_edge_label_nodes) # 0,1,3,4,5,6,7,8,9,
          
            if sampling_mode=='triplet':
                
                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges
                yield add_self_loops(hgt_batch), local_edge_label_index, edge_label, batch[target_edge].input_id
            else: # sampling_mode=='binary':
                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges, as well as original edge indices
                yield add_self_loops(hgt_batch), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id
    
    def get_hgt_2types_with_selfloops(loader):
        for batch in loader:
            if sampling_mode=='triplet':   
                original_edge_label_index_class1 = torch.LongTensor(batch[target_edge[0]].n_id[batch[target_edge[0]].src_index.unique()])
                original_edge_label_index_class2 = torch.LongTensor(batch[target_edge[2]].n_id[torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index)).unique()])
                
                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src = torch.cat((src,src), dim=1)
                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index),dim=0).unsqueeze(0)
                
                local_edge_label_index = torch.cat((src, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.shape[0])))

            elif sampling_mode=='binary':
                original_edge_label_index_class1 = batch[target_edge[0]].n_id[batch[target_edge].edge_label_index[0,:].unique()]
                original_edge_label_index_class2 = batch[target_edge[2]].n_id[batch[target_edge].edge_label_index[1,:].unique()]

            else:
                raise Exception('binary or triplet sampling mode')

            # batch the start and end supervision nodes separately
            hgt_batch1 = get_hgt(data, target_edge[0], original_edge_label_index_class1)
            hgt_batch2 = get_hgt(data, target_edge[2], original_edge_label_index_class2)
            
            
            # ** We dont need to remove any edges ** since the supervision edges wont be sampled by hgt
            if sampling_mode=='triplet':
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), local_edge_label_index, edge_label, batch[target_edge].input_id
            else: # sampling_mode=='binary':
                # we can access the corresponding nodes of edge_label_index[0,:] in hgt_batch1[target_edge[0]], those of [1,:] in hgt_batch2...
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id

        
    if target_edge[0] == target_edge[2]:
        # same edge type, only need to sample once
        return get_hgt_with_selfloops(linkNeighborLoader)
    else:
        return get_hgt_2types_with_selfloops(linkNeighborLoader)


In [1]:
import os
import torch
from torch_geometric.data import HeteroData


filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
size = os.path.getsize(filename)
print('size of dataset on disk: ', size/1e9, 'gb')

if os.path.exists(filename):
    data = HeteroData.from_dict(torch.load(filename))

/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


size of dataset on disk:  2.279761238 gb


In [23]:
from tqdm.auto import tqdm
values, counts = torch.unique(edge_idx[1,:], return_counts=True)

    

tensor([1.1307e-07, 1.1259e-07, 1.8421e-07,  ..., 1.2935e-07, 1.4430e-07,
        2.5072e-07], dtype=torch.float64)

In [32]:
keep = 0.75
total = counts.shape[0]b


In [22]:
arr = data[e].edge_attr.squeeze(1)

# Calculate the percentile value based on the threshold
threshold_value = torch.quantile(arr, 0.25)

# Create a mask for values that meet the threshold condition
mask = arr >= threshold_value

# Apply the mask to filter out values
filtered_data = arr[mask]
filtered_data.shape

torch.Size([12217189])

for skill job edges keep top k edges per job, k is  50


100%|██████████████████████████████████████████████████████████████████████████████████| 19773/19773 [01:15<00:00, 262.30it/s]


of 16289586 keep tensor(1208056)


In [59]:
d, counts = data[e].edge_index[1,:][mask.cpu()].unique(return_counts=True)

In [19]:
rev_e = (e[2],'rev_'+e[1],e[0])

IndexError: The shape of the mask [16289586] at index 0 does not match the shape of the indexed tensor [1208056, 1] at index 0

IndexError: The shape of the mask [16289586] at index 0 does not match the shape of the indexed tensor [2, 1208056] at index 1

In [26]:
data[e].edge_index.shape

torch.Size([2, 1208056])

In [ ]:
#train_data, val_data, test_data = get_datasets(get_edge_attr=False)
# testing
#input_edgetype = ('jobs', 'job_job', 'jobs')
#loader = get_hgt_linkloader(train_data, input_edgetype, 4, True, 'triplet', 1, [10], num_workers=0)
#minibatch, edge_label_index, edge_label, input_edge_ids = next(iter(loader))
#minibatch
#input_edgetype = ('skills', 'qualification_skill', 'qualifications')
#loader = get_hgt_linkloader(train_data, input_edgetype, 10, True, 'triplet', 1, [10], num_workers=0)
#minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id = next(iter(loader))
#input_edge_id


size of dataset on disk:  2.27811871 gb
loading saved heterodata object
normalizing column  773  of node type  skills
normalizing column  774  of node type  skills
normalizing column  24  of node type  people
normalizing column  773  of node type  jobs
normalizing column  774  of node type  jobs
HeteroData(
  courses_and_programs={
    x=[0, 815],
    n_id=[0],
    num_sampled_nodes=[2],
  },
  qualifications={
    x=[0, 786],
    n_id=[0],
    num_sampled_nodes=[2],
  },
  skills={
    x=[0, 775],
    n_id=[0],
    num_sampled_nodes=[2],
  },
  people={
    x=[0, 25],
    n_id=[0],
    num_sampled_nodes=[2],
  },
  jobs={
    x=[12, 775],
    n_id=[12],
    num_sampled_nodes=[2],
    src_index=[4],
    dst_pos_index=[4],
    dst_neg_index=[4],
  },
  organizations={
    x=[0, 3],
    n_id=[0],
    num_sampled_nodes=[2],
  },
  (skills, qualification_skill, qualifications)={
    edge_index=[2, 0],
    edge_label=[431],
    edge_label_index=[2, 431],
    e_id=[0],
    num_sampled_edges=

HeteroData(
  courses_and_programs={
    x=[0, 815],
    n_id=[0],
  },
  qualifications={
    x=[0, 786],
    n_id=[0],
  },
  skills={
    x=[10, 775],
    n_id=[10],
  },
  people={
    x=[10, 25],
    n_id=[10],
  },
  jobs={
    x=[22, 775],
    n_id=[22],
    input_id=[12],
    batch_size=12,
  },
  organizations={
    x=[0, 3],
    n_id=[0],
  },
  (skills, qualification_skill, qualifications)={
    edge_index=[2, 0],
    edge_label=[431],
    edge_label_index=[2, 431],
    e_id=[0],
  },
  (skills, course_and_program_skill, courses_and_programs)={
    edge_index=[2, 0],
    edge_label=[69687],
    edge_label_index=[2, 69687],
    e_id=[0],
  },
  (courses_and_programs, course_qualification, qualifications)={
    edge_index=[2, 0],
    edge_label=[567],
    edge_label_index=[2, 567],
    e_id=[0],
  },
  (courses_and_programs, course_and_programs_student, people)={
    edge_index=[2, 0],
    edge_label=[149433],
    edge_label_index=[2, 149433],
    e_id=[0],
  },
  (jobs, job_s

In [ ]:
import random

def get_minibatch_count(data, batch_size):
    batches = []
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        
    return len(batches)

def uniform_hgt_sampler(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):
    # return batches from all edgetypes with each "edge" being drawn uniformly at random (but we translate the probabilities to batches), last batches of each edge type may be smaller than batch_size
    batches = []
    loaders = {}
    # only the non-reverse edge types for now
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)
        
    random.seed(14)
    random.shuffle(batches)
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()
    
    print('total batches:', len(batches))
    
    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        yield same_nodetype, target_edge_type, next(loaders[target_edge_type])


In [ ]:
if __name__ == '__main__':
    import datetime
    import time

    # num neighbors: sample 25 one hop neighbors and for each one hop neighbor again 10 neighbors

    batch_size = 32
    num_relationships = len(train_data.edge_types)
    one_hop_neighbors = (25 * batch_size)//num_relationships # per relationship type
    two_hop_neighbors = (25 * 10 * batch_size)//num_relationships # per relationship type
    num_neighbors = [one_hop_neighbors, two_hop_neighbors]
    print('num_neighbors', num_neighbors)

    sampler = uniform_hgt_sampler(train_data, batch_size, True, 'binary', 1, num_neighbors, num_workers=0)
    start = datetime.datetime.now()
    print(start)
    print()
    for i,(same_nodetype, target_edge_type, batch) in enumerate(sampler):
        
        # batching is different depending on if node types in edge are same or different
        edge_type = batch[-1]
        if same_nodetype:
            minibatch, edge_label_index, edge_label, input_edge_ids = batch
            print(minibatch)
        else:
            minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id = batch
            print(minibatchpart1)
            
        print(i,target_edge_type)
        
        break
        time.sleep(5)
        
    end = datetime.datetime.now()
    print()
    print(end-start)

num_neighbors [36, 363]
2023-11-01 14:21:24.226340



TypeError: get_hgt_linkloader() missing 1 required positional argument: 'num_workers'